In [30]:
import pandas as pd

data = pd.read_csv('competition_data.csv')
submission = pd.read_csv('submission.csv')

In [31]:
#eliminamos datos particulares de Chona, son siempre los mismos, y los que consideramos irrelevanets
data = data.drop(columns=['username', 'conn_country', 'user_agent_decrypted'])

In [32]:
# del timestamp, solo nos interesa la hora, el dia de la semana y el mes

data['ts'] = pd.to_datetime(data['ts'])

# Extraer la hora
data['hour'] = data['ts'].dt.hour

# Extraer el día de la semana (0 es lunes, 6 es domingo)
data['day_of_week'] = data['ts'].dt.dayofweek

# Extraer el mes
data['month'] = data['ts'].dt.month

# dropeamos ts
data = data.drop(columns=['ts'])

In [33]:
# del dispositivo, solo queremos saber si se reprodujo en celular (iphone) o no
data['is_iphone'] = data['platform'].apply(lambda x: 1 if ('ios' in x or 'iOS' in x) else 0)

# dropeamos platform
data = data.drop(columns=['platform'])

In [34]:
data

,Unnamed: 0,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,shuffle,TARGET,hour,day_of_week,month,is_iphone
0,110163,Crazy For U,Big Time Rush,24/seven,spotify:track:3jFfr89lnSmb4QBtfG8JBP,clickrow,False,False,5,6,3,1
1,66026,Nada Personal - Remasterizado 2007,Soda Stereo,Me Verás Volver (Hits & Más),spotify:track:09TTeexnlKewZdjOak2sV2,trackdone,True,False,10,0,6,1
2,116790,Good Times,CHIC,Risqué,spotify:track:0G3fbPbE1vGeABDEZF0jeG,trackdone,True,True,2,6,7,1
3,18431,Verano del 92,Los Piojos,3er Arco,spotify:track:1NXvuBAq48QrxRFQZVmORQ,trackdone,False,True,4,6,9,1
4,82941,Simpatico,Ekko Park,Simpatico,spotify:track:2gYJY0sIx1ErgTIha2nPRg,trackdone,True,False,18,3,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...
100139,78756,Losing My Religion,R.E.M.,In Time: The Best Of R.E.M. 1988-2003,spotify:track:12axV6NUqaYH3yFUWwArzr,trackdone,True,False,20,3,4,1
100140,12585,Cerca De La Revolucion,Charly García,Piano Bar,spotify:track:66grIvFLGrI4tNhggO2DAd,trackdone,True,False,21,2,11,1
100141,93960,A Los Jóvenes De Ayer - Remastered 2012,Serú Girán,Bicicleta,spotify:track:6YAl320SfxLO4rVIZTFKqG,trackdone,True,False,1,4,1,1
100142,74339,heat not hot,Serengeti,heat not hot,spotify:track:0rKLO1hXpmoIthQgJKoczN,trackdone,True,False,17,2,5,1


In [35]:
# Aplicar One-Hot Encoding a razon de reproduccion
reason_start_encoded = pd.get_dummies(data['reason_start'], prefix='reason', dummy_na=True)

# Concatenar el DataFrame original con el nuevo DataFrame de variables codificadas
data = pd.concat([data, reason_start_encoded], axis=1)

# Dropear la columna original
data = data.drop(columns=['reason_start'])

In [38]:
# Lista de columnas de One-Hot Encoding creadas previamente
encoded_columns = reason_start_encoded.columns


# convertir a entero los bools
for col in encoded_columns:
    data[col] = data[col].astype(int)

In [39]:
data

,Unnamed: 0,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,shuffle,TARGET,hour,day_of_week,month,...,reason_appload,reason_backbtn,reason_clickrow,reason_fwdbtn,reason_playbtn,reason_remote,reason_trackdone,reason_trackerror,reason_unknown,reason_nan
0,110163,Crazy For U,Big Time Rush,24/seven,spotify:track:3jFfr89lnSmb4QBtfG8JBP,False,False,5,6,3,...,0,0,1,0,0,0,0,0,0,0
1,66026,Nada Personal - Remasterizado 2007,Soda Stereo,Me Verás Volver (Hits & Más),spotify:track:09TTeexnlKewZdjOak2sV2,True,False,10,0,6,...,0,0,0,0,0,0,1,0,0,0
2,116790,Good Times,CHIC,Risqué,spotify:track:0G3fbPbE1vGeABDEZF0jeG,True,True,2,6,7,...,0,0,0,0,0,0,1,0,0,0
3,18431,Verano del 92,Los Piojos,3er Arco,spotify:track:1NXvuBAq48QrxRFQZVmORQ,False,True,4,6,9,...,0,0,0,0,0,0,1,0,0,0
4,82941,Simpatico,Ekko Park,Simpatico,spotify:track:2gYJY0sIx1ErgTIha2nPRg,True,False,18,3,7,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100139,78756,Losing My Religion,R.E.M.,In Time: The Best Of R.E.M. 1988-2003,spotify:track:12axV6NUqaYH3yFUWwArzr,True,False,20,3,4,...,0,0,0,0,0,0,1,0,0,0
100140,12585,Cerca De La Revolucion,Charly García,Piano Bar,spotify:track:66grIvFLGrI4tNhggO2DAd,True,False,21,2,11,...,0,0,0,0,0,0,1,0,0,0
100141,93960,A Los Jóvenes De Ayer - Remastered 2012,Serú Girán,Bicicleta,spotify:track:6YAl320SfxLO4rVIZTFKqG,True,False,1,4,1,...,0,0,0,0,0,0,1,0,0,0
100142,74339,heat not hot,Serengeti,heat not hot,spotify:track:0rKLO1hXpmoIthQgJKoczN,True,False,17,2,5,...,0,0,0,0,0,0,1,0,0,0


In [37]:
#pip install spotipy